### Copy and compress video files

In [1]:
from pathlib2 import Path
import pathlib2 
import os

In [2]:
from tqdm import tqdm

In [3]:
# settings
crf = 15 # 0 = lossless, 51 = worst

In [4]:
# mirror flip bottom cameras 
cameras_to_mirror = []#["FB", "BB"]

In [5]:
base_path_local = Path("/media/dattalab/starling_rec/recordings/")
base_path_save= Path('/n/groups/datta/tim_sainburg/datasets/3D_keypoints/compressed/') 
subfolder = '22-10-11-test-flicker'

In [20]:
# '/home/dattalab/JARVIS/sync_flir_azure_9_27/recordings/Recording_2022-09-27T122406'
# folder with recordings
video_folder = csv_folder = base_path_local / subfolder
# folder to save recordings folder
output_folder = base_path_save / subfolder

In [7]:
if not os.path.exists('/n/groups/datta/tim_sainburg'):
    raise ValueError('must mount /n/groups/datta')

In [8]:
save_folder = output_folder /video_folder.stem

In [9]:
def ensure_dir(file_path):
    """ create a safely nested folder
    """
    if type(file_path) == str:
        if "." in os.path.basename(os.path.normpath(file_path)):
            directory = os.path.dirname(file_path)
        else:
            directory = os.path.normpath(file_path)
        if not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except FileExistsError as e:
                # multiprocessing can cause directory creation problems
                print(e)
    elif type(file_path) == pathlib2.PosixPath:
        # if this is a file
        if len(file_path.suffix) > 0:
            file_path.parent.mkdir(parents=True, exist_ok=True)
        else:
            file_path.mkdir(parents=True, exist_ok=True)

In [10]:
ensure_dir(save_folder)

In [11]:
videos_to_convert = list(video_folder.glob('**/*.avi'))

In [12]:
list(videos_to_convert)

[PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/T.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/BR.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/BB.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/BL.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/FB.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/FR.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/Recording_2022-11-03T174548/FL.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/T.avi'),
 PosixPath('/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/BR

In [13]:
import ffmpeg
import numpy as np
from tqdm.auto import tqdm
import subprocess

In [14]:
#ffmpeg -i FL.avi -c:v libx265 -vtag hvc1 -crf 20 -preset ultrafast FL_compressed-265-2.avi

In [17]:
from IPython.display import clear_output
import os
import shutil


In [18]:
delete_local_video = False

In [22]:
# copy all metadata (metadata.csv and triggerdata.csv)
metadata_to_copy = list(csv_folder.glob('**/metadata.csv'))
triggerdata_to_copy = list(csv_folder.glob('**/triggerdata.csv'))
for csv_file in tqdm(metadata_to_copy + triggerdata_to_copy):
    # get the name to save for the output folder
    number_of_parents_to_base = np.where([i == csv_folder for i in list(csv_file.parents)])[0][0]
    sub_folders = csv_file.parts[len(csv_file.parts)-1-number_of_parents_to_base:-1]
    csv_out = output_folder
    for i in sub_folders:
        csv_out = csv_out / i
    csv_out = csv_out / csv_file.name
    
    ensure_dir(csv_out)
    
    if csv_out.exists():
        print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))

        #if delete_local_video:
            #os.remove(csv_file)

        continue
        
    else:
        shutil.copyfile(csv_file, csv_out)

  0%|          | 0/23 [00:00<?, ?it/s]

In [25]:
for video in tqdm(videos_to_convert):
    try:
        # get the name to save for the output folder
        number_of_parents_to_base = np.where([i == video_folder for i in list(video.parents)])[0][0]
        sub_folders = video.parts[len(video.parts)-1-number_of_parents_to_base:-1]
        video_out = output_folder
        for i in sub_folders:
            video_out = video_out / i
        video_out = video_out / video.name

        ensure_dir(video_out)

        if video_out.exists():
            print('Video already exists, skipping \n\t {}'.format('/'.join(video.parts[-3:])))
            
            if delete_local_video:
                os.remove(video)
            
            continue

        print('converting {}'.format('/'.join(video.parts[-3:])))
        #continue
        # https://www.programcreek.com/python/example/117479/ffmpeg.output

        flip_video = video.stem in cameras_to_mirror

        ffmpeg_command = (
            ffmpeg
            .input(video.as_posix())

        )
        if flip_video:
            ffmpeg_command = ffmpeg_command.video.hflip()
        ffmpeg_command = (
            ffmpeg_command.output(
                video_out.as_posix(),
                vcodec = "libx265", 
                crf = crf, 
                vtag="hvc1", 
                preset="ultrafast")
        )
        out, _ = ffmpeg_command.run()
        
        
        # if out is successful 
        if delete_local_video:
            os.remove(video)
        
        #out, _ = (
        #    ffmpeg
        #    .input(video.as_posix())
        #    .output(video_out.as_posix(), vcodec = "libx265", crf = crf, vtag="hvc1", preset="ultrafast")
        #    .run()
        #)
    
        
    except KeyError:
        # delete bad video file
        if video_out.exists():
            os.remove(video_out)
        break
        
    except Exception:
        # delete bad video file
        if video_out.exists():
            os.remove(video_out)
        
        clear_output(wait=True)
        print('BACKUP FAILED {}'.format(video))
        continue

  0%|          | 0/84 [00:00<?, ?it/s]

Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/BR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/BB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/BL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/FB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/FR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/Recording_2022-11-03T174548/FL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/BR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/BB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M3/BL.a

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

converting 22-10-11-test-flicker/2022_11_3_markerset/BR.avi


Input #0, avi, from '/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/2022_11_3_markerset/BR.avi':
  Metadata:
    encoder         : Lavf58.29.100
  Duration: 00:36:25.00, start: 0.000000, bitrate: 173321 kb/s
    Stream #0:0: Video: mjpeg (Baseline) (MJPG / 0x47504A4D), yuvj420p(pc, bt470bg/unknown/unknown), 1280x1024 [SAR 1:1 DAR 5:4], 173308 kb/s, 100 fps, 100 tbr, 100 tbn, 100 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (mjpeg (native) -> hevc (libx265))
Press [q] to stop, [?] for help
x265 [info]: HEVC encoder version 3.2.1+1-b5c86a64bbbe
x265 [info]: build info [Linux][GCC 9.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4.1 (Main tier)
x265 [info]: Thread pool created using 24 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(32 rows)
x265 [info]: Coding QT: max CU size, min CU size : 32 / 16
x26

...

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/BR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/BB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/BL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/FB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/FR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M2/FL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_M1/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_M1/BR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_M1/BB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_M1/BL.avi
Video already exists, 

Input #0, avi, from '/media/dattalab/starling_rec/recordings/22-10-11-test-flicker/ketamine-5m/FB.avi':
  Metadata:
    encoder         : Lavf58.29.100
  Duration: 00:02:45.84, start: 0.000000, bitrate: 68030 kb/s
    Stream #0:0: Video: mjpeg (Baseline) (MJPG / 0x47504A4D), yuvj420p(pc, bt470bg/unknown/unknown), 1280x1024, 68008 kb/s, 100 fps, 100 tbr, 100 tbn, 100 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (mjpeg (native) -> hevc (libx265))
Press [q] to stop, [?] for help
x265 [info]: HEVC encoder version 3.2.1+1-b5c86a64bbbe
x265 [info]: build info [Linux][GCC 9.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4.1 (Main tier)
x265 [info]: Thread pool created using 24 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(32 rows)
x265 [info]: Coding QT: max CU size, min CU size : 32 / 16
x265 [info]: Residual QT: max T

Video already exists, skipping 
	 22-10-11-test-flicker/ketamine-5m/FR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/ketamine-5m/FL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/BR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/BB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/BL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/FB.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/FR.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVLightsTest_11-1_Mouse1/FL.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M1/T.avi
Video already exists, skipping 
	 22-10-11-test-flicker/UVFlashingLightsTest_11-1_M1/BR.avi
Video already exists, skipping 
	 22-10-11-test-flic

frame=16584 fps=257 q=-0.0 Lsize=  339969kB time=00:02:45.82 bitrate=16795.5kbits/s speed=2.57x    
video:339566kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.118504%
x265 [info]: frame I:     67, Avg QP:22.51  kb/s: 53932.32
x265 [info]: frame P:   4756, Avg QP:24.94  kb/s: 2059.09 
x265 [info]: frame B:  11761, Avg QP:26.63  kb/s: 22498.10
x265 [info]: consecutive B-frames: 0.3% 9.7% 35.8% 54.2% 

encoded 16584 frames in 64.51s (257.09 fps), 16763.55 kb/s, Avg QP:26.13
